In [15]:
import numpy as np
import pandas as pd
from astropy.wcs import WCS
from astropy.io import fits
import matplotlib.pyplot as plt


In [6]:
goodss_result = pd.read_csv('/Users/neal/Documents/PhD/projects/SED-fitting/Clump/results/catalog_n/goodsslast.csv')

In [12]:
def load_image(galaxy_id, band):
    # the base path where the FITS files 
    base_path = '/Users/neal/Documents/PhD/projects/SED-fitting/Clump/Cutouts_galaxies'
    
    # Construct the filename using the galaxy ID and band
    file_name = f'{base_path}/goodss_{galaxy_id}_{band}_images.fits'
    
    # Open the FITS file and return the image data
    with fits.open(file_name) as hdul:
        image_data = hdul[0].data
    
    return image_data

In [16]:
# Loop over the catalog
for index, row in goodss_result.iterrows():
    if row['Clump_ID'] != -99:  # Ensure this is an actual detected clump
        
        # Map redshift to band directly
        if 0.5 <= row['z'] < 1:
            band = 'f275w'
        elif 1 <= row['z'] < 2:
            band = 'f435w'
        elif 2 <= row['z'] <= 3:
            band = 'f606w'
        else:
            raise ValueError("Redshift out of bounds for band selection")
        
        image_data = load_image(row['CANDELS_ID'], band)

In [17]:
def apply_mask(image, clump_positions, current_index, mask_radius=3):
    mask = np.ones_like(image, dtype=bool)
    for i, pos in enumerate(clump_positions):
        if i != current_index:
            x, y = pos
            mask[max(0, y - mask_radius):min(image.shape[0], y + mask_radius + 1),
                 max(0, x - mask_radius):min(image.shape[1], x + mask_radius + 1)] = False
    return image * mask

In [18]:
def centroid(im, region=None):
    """Compute centroid of image or region (M10/M00, M01/M00) (x, y)."""
    if region is not None:
        im = im[region[0]:region[1], region[2]:region[3]]
        offset = [region[2], region[0]]
    else: 
        offset = [0, 0]
    m00 = np.sum(im)
    m10 = np.sum(np.sum(im, axis=1) * np.arange(im.shape[0]))
    m01 = np.sum(np.sum(im, axis=0) * np.arange(im.shape[1]))
    return [m10 / m00 + offset[0], m01 / m00 + offset[1]]


In [9]:
mask_radius = 3

In [19]:
goodss_result['new_X_centroid'] = np.nan
goodss_result['new_Y_centroid'] = np.nan


In [ ]:
# Assuming your DataFrame has 'X_clump' and 'Y_clump' columns with initial clump positions

for index, row in goodss_result.iterrows():
    if row['Clump_ID'] != -99:  # Exclude entries without a detected clump
        # Get the appropriate band based on redshift
        band = get_band_from_redshift(row['z'])
        
        # Load the image data for the current clump
        image_data = load_image(row['CANDELS_ID'], band)

        # Get all clump positions for the current galaxy (excluding the current clump)
        clump_positions = [(x, y) for idx, (x, y) in enumerate(zip(goodss_result['X_clump'], goodss_result['Y_clump'])) if idx != index]

        # Apply the mask to the image data
        masked_image = apply_mask(image_data, clump_positions, index, mask_radius=3)

        # Define the region around the current clump for centroid calculation
        region = [int(max(0, row['Y_clump'] - 3)), int(min(image_data.shape[0], row['Y_clump'] + 4)),
                  int(max(0, row['X_clump'] - 3)), int(min(image_data.shape[1], row['X_clump'] + 4))]
        
        # Compute the centroid of the masked clump
        new_centroid = centroid(masked_image, region=region)

        # Update the DataFrame with the new centroid coordinates
        goodss_result.at[index, 'new_X_centroid'] = new_centroid[0]
        goodss_result.at[index, 'new_Y_centroid'] = new_centroid[1]


In [ ]:
updated_catalog_path = '/Users/neal/Documents/PhD/projects/SED-fitting/Clump/results/newcenters_goodsslast.csv'
goodss_result.to_csv(updated_catalog_path, index=False)


In [10]:
# # Process each clump to find the new centers
# for index, row in goodss_result.iterrows():
#     if row['Clump_ID'] != -99:
#         image_data = load_image(row['CANDELS_ID'], appropriate_band_based_on_z(row['z']))
#         clump_positions = [(x, y) for x, y in zip(goodss_result['RA_clump'], goodss_result['DEC_clump'])]
#         current_position = (row['RA_clump'], row['DEC_clump'])
        
#         # Apply the mask to the image to ignore other clumps
#         masked_image = apply_mask(image_data, clump_positions, index, mask_radius)

#         # Define the region for centroid calculation directly here
#         region = (max(0, int(current_position[1]) - mask_radius), 
#                   min(masked_image.shape[0], int(current_position[1]) + mask_radius + 1),
#                   max(0, int(current_position[0]) - mask_radius), 
#                   min(masked_image.shape[1], int(current_position[0]) + mask_radius + 1))

#         # Calculate the centroid for the current clump
#         new_centroid = centroid(masked_image, region=region)

#         # Update the DataFrame with the new centroid values
#         goodss_result.at[index, 'RA_clump'] = new_centroid[0]
#         goodss_result.at[index, 'DEC_clump'] = new_centroid[1]

# # Save the updated results
# goodss_result.to_csv(result_sample_fname, index=False)

NameError: name 'load_image' is not defined